In [1]:
few_shot_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.train.short.csv'
data_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.smalltest.csv'
output_dir = '/scratch/dzhang5/LLM/TWEET-FID/test-results-autolabel-ner'
model_name = "refuel-llm"
label_column = 'CategorizedLabels'
text_column = 'tweet'
example_selection_label_column = 'sentence_class'
few_shot_num = 8
few_shot_selection = "semantic_similarity"
token_path = "/home/dzhang5/.cache/huggingface/token"
cache=False
console_output=True
temperature=0.1

In [3]:
from autolabel.schema import ModelProvider, TaskType
from autolabel.models import register_model, MODEL_REGISTRY
from hf_pipeline_new import HFPipelineLLMNew
from few_shot_new import NewAutoLabelConfig, NewExampleSelectorFactory
from autolabel.few_shot import ExampleSelectorFactory
from template_inst import update_inst_mode
from named_entity_recognition_new import NewNamedEntityRecognitionTask
from classification_new import NewClassificationTask
from autolabel.tasks import TASK_TYPE_TO_IMPLEMENTATION 
import sys

In [4]:
update_inst_mode(model_name)
TASK_TYPE_TO_IMPLEMENTATION[TaskType.NAMED_ENTITY_RECOGNITION] = NewNamedEntityRecognitionTask
TASK_TYPE_TO_IMPLEMENTATION[TaskType.CLASSIFICATION] = NewClassificationTask
sys.modules['autolabel.labeler'].ExampleSelectorFactory = NewExampleSelectorFactory
register_model(ModelProvider.HUGGINGFACE_PIPELINE, HFPipelineLLMNew)

In [4]:
from autolabel import LabelingAgent, AutolabelDataset
import json
import os
import pandas as pd

In [5]:
with open(token_path) as tfile:
    token_str = tfile.read()

from huggingface_hub import login
login(token=token_str)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dzhang5/.cache/huggingface/token
Login successful


In [6]:
no_auto = ["microsoft/prophetnet", "microsoft/phi-2", "google/pegasus-x"]
if any([pre in model_name for pre in no_auto]):
    device_map = None
else:
    device_map = "auto"

In [7]:
refuel_models = ["refuel-llm", "llama-13b-chat"]
if model_name in refuel_models:
    provider = "refuel"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetNER_{few_shot_selection}_{model_name}"
elif model_name.startswith('gpt'):
    provider = "openai"
    em_provider = "openai"
    model_params = {"max_tokens":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetNER_{few_shot_selection}_{model_name}"
else:
    provider = "huggingface_pipeline"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature,
                    "quantize": 16, "device_map": device_map,
                    "token": token_str}
    task_name = f"FoodborneIllnessIncidentTweetNER_{few_shot_selection}_{model_name.split('/')[1]}"

In [8]:
if not os.path.exists(output_dir):
    # Create the directory
    os.makedirs(output_dir)
output_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_' + os.path.split(data_path)[-1]
output_path = os.path.join(output_dir, output_name)

In [9]:
label_description = {"Food": "are specific food item that caused the potential foodborne illness incident.",
                     "Symptom": "are specific symptoms experienced by the affected person as a result of the suspected foodborne illness.",
                     "Location": "pretrain to the location where the affected person purchased or acquired the food associated with the potential foodborne illness.",
                     "Keyword": """include other relevant keywords or terms associated with foodborne illnesses, such as "food poisoning"."""}

entity_description = ""
for k, v in label_description.items():
    entity_description += f"{k} entities {v}\n"

task_guideline = ("You are an expert at extracting Food, Location, Symptom, and Keyword entities that are related to foodborne illness incident from text. " 
                  "Within the TWEET-FID dataset, your task is to extract entities mentioned in text, and classify them into one of the following categories.\nCategories:\n{labels}\n"
                  "Below are the defintions of each type of entity:\n")
task_guideline += entity_description          
task_guideline += "Use the following examples as a guide for your predictions and format your responses similarly."

In [10]:
test_data = pd.read_csv(data_path)

In [11]:
config = {
    "task_name": task_name,
    "task_type": "named_entity_recognition",
    "dataset": {
        "label_column": label_column,
        "text_column": text_column,
        "example_selection_label_column": example_selection_label_column,
        "delimiter": ",",
        "label_description": label_description 
    },
    "model": {
        "provider": provider,
        "name": model_name,
        "params": model_params
    },
    "embedding": {
        "provider": em_provider,
    },
    "prompt": {
        "task_guidelines": task_guideline,
        "output_guidelines": "You will answer with just the correct entities alongwith their labels and nothing else. Note that if the given text does not include any entities related to foodborne illness, you will not answer with any word.",
        "labels": [
            "Food",
            "Symtom",
            "Location",
            "Keyword"
        ],
        "few_shot_examples": few_shot_path,
        "few_shot_selection": few_shot_selection,
        "few_shot_num": few_shot_num,
        "example_template": f"Input: {{{text_column}}}\nOutput: {{{label_column}}}"
    }
}

config = NewAutoLabelConfig(config)

In [12]:
agent = LabelingAgent(config=config, console_output=console_output, cache=cache)

2024-01-27 22:38:54 autolabel.labeler WARNING: cache parameter is deprecated and will be removed soon. Please use generation_cache and transform_cache instead.


In [13]:
ds = AutolabelDataset(test_data[[text_column, example_selection_label_column, label_column]], config=config)
agent.plan(ds)

2024-01-27 22:38:55 sentence_transformers.SentenceTransformer INFO: Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
2024-01-27 22:38:57 sentence_transformers.SentenceTransformer INFO: Use pytorch device: cuda


Batches:   0%|          | 0/78 [00:00<?, ?it/s]

Output()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

┌──────────────────────────┬──────┐
│ Total Estimated Cost     │ $0.0 │
│ Number of Examples       │ 10   │
│ Average cost per example │ $0.0 │
└──────────────────────────┴──────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

<s>[INST] <<SYS>>
    You are an expert at extracting Food, Location, Symptom, and Keyword entities that are related to foodborne 
illness incident from text. Within the TWEET-FID dataset, your task is to extract entities mentioned in text, and 
classify them into one of the following categories.
Categories:
Food
Symtom
Location
Keyword
Below are the defintions of each type of entity:
Food entities are specific food item that caused the potential foodborne illness incident.
Symptom entities are specific symptoms experienced by the affected person as a result of the suspected foodborne 
illness.
Location entities pretrain to the location where the affected person purchased or acquired the food associated with
the potential foodborne illness.
Keyword entities include other relevant keywords or terms associated with foodborne illnesses, such as "food 
poisoning".
Use the following examples as a guide for your predictions and format your responses similarly.You will answer with
just the correct entities alongwith their labels and nothing else. Note that if the given text does not include any
entities related to foodborne illness, you will not answer with any word.
Input: Think I might have gotten a mild case of food-poisoning today - so that's been super fun.
Output: 

Input: @USER To be honest, it's been a little rough. Food poisoning. Things are looking up, though.
Output: Food poisoning%Keyword

Input: @USER LOL I know! I've had a stomach bug or food poisoning since yesterday afternoon
Output: food poisoning%Keyword

Input: @USER Hoping you are feeling better tonight! Food poisoning is rough! See you Tues. night! TAKE CARE OF 
YOURSELF!!
Output: Food poisoning%Keyword

Input: Literally have been in bed all day bc I have food poisoning
Output: food poisoning%Keyword

Input: @USER Awwwww Coo , Don't Look Like I'm Doing Anything Today Got Food Poison ;(
Output: Food Poison%Keyword

Input: @USER I heard that you got food poisoning. That sucks. I'm not doing well. I'm having a really bad day. 
[EMOJI_pensive_face]
Output: food poisoning%Keyword

Input: I've been in the hospital all night because of food poison I finally get home and watch @USER thinking I'd 
be happy. I'm not happy![EMOJI_face_with_tears_of_joy]
Output: food poison%Keyword
    <<SYS>>
    Input: @USER As much fun as I can. Woke up with food poisoning or stomach flu. Been bugging me all day #tmi 
Almost done driving for the day
Output: [/INST]

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [14]:
# now, do the actual labeling
ds = agent.run(ds)

Output()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Actual Cost: 0.0

┏━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ support ┃ completion_rate ┃ f1_exact ┃ f1_strict ┃ f1_partial ┃ f1_ent_type ┃ accuracy ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ 10      │ 1.0             │ 0.3      │ 0.2       │ 0.3        │ 0.2         │ 0.1176   │
└─────────┴─────────────────┴──────────┴───────────┴────────────┴─────────────┴──────────┘

In [15]:
metrics = ds.eval()

┏━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ support ┃ completion_rate ┃ f1_exact ┃ f1_strict ┃ f1_partial ┃ f1_ent_type ┃ accuracy ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ 10      │ 1.0             │ 0.3      │ 0.2       │ 0.3        │ 0.2         │ 0.1176   │
└─────────┴─────────────────┴──────────┴───────────┴────────────┴─────────────┴──────────┘

In [16]:
ds.df.to_csv(output_path, index=False)
ds.df.to_pickle(output_path.replace('.csv', '.pkl'))